In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
%load_ext autoreload
%autoreload 2

In [5]:
from dotenv import load_dotenv
import openai


load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup as bs, NavigableString, Comment

In [36]:
HEADLESS = False
MODEL = 'GPT3.5'
PREPROCESS = None # None | 'XPATH' | 'SPANS' | 'ATTRIBUTES'
TESTS_LIST_SPEC = 'tests.sample.json'

In [11]:
chrome_options = Options()
if HEADLESS:
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--window-size=3072x1920");

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

if not HEADLESS:
    driver.maximize_window()

In [32]:
import json


def create_test_list():
    test_list_spec = {}
    test_list = []

    with open(TESTS_LIST_SPEC, 'r') as f:
        test_list_spec = json.load(f)
        f.close()
    
    
    for app, tests in test_list_spec.items():
        # include all forms in the application
        if tests[0] == '*':
            all_app_tests = filter(lambda x: 'json' in x, os.listdir(f'dataset/{app}'))
            all_app_tests_locs = list(map(lambda x: f'dataset/{app}/{x}', all_app_tests))
            test_list.extend(all_app_tests_locs)
            continue
        app_tests_locs = map(lambda x: f'dataset/{app}/{x}.json', tests)
        test_list.extend(app_tests_locs)

    test_jsons = []
    
    for test in test_list:
        with open(test, 'r') as f:
            test_jsons.append(json.load(f))
        f.close()
    
    return test_jsons

In [34]:
test_list = create_test_list()

In [38]:
from method.models import chat_gpt_plain

In [39]:
import time


def preprocess_form(form):
    if PREPROCESS == None:
        return form.get_attribute('outerHTML')
    
    return form.get_attribute('outerHTML')


def run_method(form):
    processed = preprocess_form(form)
    res = chat_gpt_plain(processed, openai.api_key)
    return res


def run_test(test):
    driver.get(test['url'])
    time.sleep(1)
    
    form = driver.find_element(By.XPATH, test['xpath'])
    return 

In [40]:
processed = preprocess_form(form)

NameError: name 'form' is not defined

KeyError: 'xpath'

In [42]:
driver.get(test_list[0]['url'])

In [44]:
form = driver.find_element(By.XPATH, test_list[0]['form_xpath'])

In [46]:
processed = preprocess_form(form)

In [52]:
res = chat_gpt_plain(processed, openai.api_key)

  Prompt: [
  {
    "role": "system",
    "content": "\nYou are FormGPT, an AI assistant embedded in a crawler that has the task of generating different values for form inputs.\nYour decisions must always be made independently without seeking user assistance. Play to your strengths as an LLM and pursue simple strategies.\n\nGOALS:\n1. Given a form HTML, understand the purpose of the form, and generate values for each input in the form.\n\nCONSTRAINTS:\n1. No user assistance\n\nPerformance Evaluation:\n1. Continuously review and analyze your actions to ensure you are performing to the best of your abilities.\n2. Constructively self-criticize your big-picture behavior constantly.\n3. Reflect on past decisions and strategies to refine your approach.\n4. Every command has a cost, so be smart and efficient. Aim to complete tasks in the least number of steps.\n\nYou can issue these commands:\n    - FILL----X----\"Y\": Fill the <input> or <textarea> element that has a relative xpath X with va

In [54]:
print(res)

[
    {
        "command": "FILL",
        "xpath": "//input[@id='basic_username']",
        "value": "myusername"
    },
    {
        "command": "FILL",
        "xpath": "//input[@id='basic_password']",
        "value": "mypassword"
    },
    {
        "command": "CLICK",
        "xpath": "//input[@id='basic_remember']"
    },
    {
        "command": "CLICK",
        "xpath": "//button[@type='submit']"
    }
]
